In [ ]:
from torchvision.models import resnet18
from torchcam.methods import ScoreCAM
import matplotlib.pyplot as plt
from torchcam.utils import overlay_mask

# /dark_corner/ben/ISIC_0000011.jpg /dark_corner/mel/ISIC_0000031.jpg /gel_bubble/ben/ISIC_0015311.jpg

def vis_grad(model):
  model.eval()
  # Get your input
  img = read_image(data_root + '/gel_bubble/ben/ISIC_0011223.jpg')
  # Preprocess it for your chosen model
  input_tensor = normalize(resize(img, (224, 224)) / 255., [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

  with ScoreCAM(model) as cam_extractor:
    # Preprocess your data and feed it to the model
    out = model(input_tensor.to(device).unsqueeze(0))
    # Retrieve the CAM by passing the class index and the model output
    activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)

    # Resize the CAM and overlay it
    result = overlay_mask(to_pil_image(img), to_pil_image(activation_map[0].squeeze(0), mode='F'), alpha=0.5)
    # Display it
    plt.imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()

vis_grad(model_trained)

In [ ]:
def reshape_transform(tensor, height=14, width=14):
    result = tensor[:, 1:, :].reshape(tensor.size(0),
                                      height, width, tensor.size(2))

    # Bring the channels to the first dimension,
    # like in CNNs.
    result = result.transpose(2, 3).transpose(1, 2)
    return result


target_layers = [model_trained.featurizer.network.blocks[-1].norm1]
cam = ScoreCAM(model=model_trained,
                target_layers=target_layers,
                reshape_transform=reshape_transform)

targets=None
# Preprocess it for your chosen model
rgb_img = cv2.imread(data_root + '/dark_corner/ben/ISIC_0000011.jpg', 1)[:, :, ::-1]
rgb_img = cv2.resize(rgb_img, (224, 224))
rgb_img = np.float32(rgb_img) / 255
input_tensor = preprocess_image(rgb_img, mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

grayscale_cam = cam(input_tensor=input_tensor,
                        targets=targets,
                        eigen_smooth=False,
                        aug_smooth=False)
grayscale_cam = grayscale_cam[0, :]

cam_image = show_cam_on_image(rgb_img, grayscale_cam)
plt.imshow(cam_image)